# Task - 4

In [1]:
import numpy as np

from qiskit.circuit import QuantumRegister, ClassicalRegister, QuantumCircuit, CircuitInstruction
from qiskit.transpiler.coupling import CouplingMap
from qiskit.circuit.library import standard_gates
from qiskit.circuit.exceptions import CircuitError

In [2]:
# creating dictionary for labelling all gates

gates_label = {'id': standard_gates.IGate,'x': standard_gates.XGate,'y': standard_gates.YGate,'z': standard_gates.ZGate,
               'h': standard_gates.HGate, 'sx': standard_gates.SXGate,'rz': standard_gates.RZGate,'r': standard_gates.RGate,
               'p': standard_gates.PhaseGate,'rx': standard_gates.RXGate,'ry': standard_gates.RYGate,'s': standard_gates.SGate,
               'sdg': standard_gates.SdgGate,'sxdg': standard_gates.SXdgGate,'t': standard_gates.TGate,'tdg': standard_gates.TdgGate,
               'u': standard_gates.UGate,'u1': standard_gates.U1Gate,'u2': standard_gates.U2Gate,'u3': standard_gates.U3Gate,
               'cx': standard_gates.CXGate, 'dcx': standard_gates.DCXGate, 'cp': standard_gates.CPhaseGate,'crx': standard_gates.CRXGate,
               'cry' : standard_gates.CRYGate, 'crz': standard_gates.CRZGate, 'csx': standard_gates.CSXGate,'cu': standard_gates.CUGate,
               'cu1': standard_gates.CU1Gate, 'cy': standard_gates.CYGate, 'cz' : standard_gates.CZGate, 'rxx': standard_gates.RXXGate,
               'ryy': standard_gates.RYYGate, 'rzz': standard_gates.RZZGate, 'rzx' : standard_gates.RZXGate, 'xx-yy': standard_gates.XXMinusYYGate,
               'xx+yy': standard_gates.XXPlusYYGate,'ecr': standard_gates.ECRGate, 'cs' : standard_gates.CSGate, 'csdg': standard_gates.CSdgGate,
               'swap':standard_gates.SwapGate,'iswap': standard_gates.iSwapGate}

In [3]:
# creating default gate set

default_gate_set = set(['id', 'u1', 'u2', 'u3', 'x', 'y', 'z', 'h', 's', 'sdg', 't', 'tdg',
                'rx', 'ry', 'rz', 'cx'])

# creating sets containing labels of one qubit and two qubit operators

one_q_ops_label = set(['id', 'u1', 'u2', 'u3', 'x', 'y', 'z', 'h', 's', 'sdg', 't', 'tdg', 'rx', 'ry', 'rz'])
two_q_ops_label = set(['cx'])   # only considering cx gates other two qubit gates can change the depth after transpilation, but one can modify to add any two qubit operations that don't require parameters
one_param_label = set(['u1', 'rx', 'ry', 'rz']) # only one qubit operations that require single parameter considered but it could be two qubit operation by some modifications in the code 
two_param_label = set(['u2'])
three_param_label = set(['u3'])

In [4]:
# function to create random quantum circuit

def RQC(num_qubits:int = 1, depth: int = 1, basis_gates :list = default_gate_set, seed :int = np.random.seed(), coupling_map = None):

    one_q_ops = [gates_label[name] for name in one_q_ops_label & set(basis_gates)]
    two_q_ops = [gates_label[name] for name in two_q_ops_label & set(basis_gates)]
    one_param = [gates_label[name] for name in one_param_label & set(basis_gates)]
    two_param = [gates_label[name] for name in two_param_label & set(basis_gates)]
    three_param = [gates_label[name] for name in three_param_label & set(basis_gates)]

    if len(one_q_ops) and len(two_q_ops)== 0:
        raise CircuitError("No gates are available to build circuit")
    
    qreg = QuantumRegister(num_qubits, 'q')
    qc = QuantumCircuit(num_qubits)

    # setting a default coupling map
    if coupling_map == None:
        coupling_map = CouplingMap.from_full(num_qubits=num_qubits) # all the qubits are fully connected

    rng = np.random.default_rng(seed = seed)

    for _ in range(depth):
        qubits = coupling_map.physical_qubits
        edges = coupling_map.get_edges()
        allow_two_q_op = bool(edges)
        while qubits:
            max_possible_operands = min(len(qubits),2) if allow_two_q_op else 1 # here we consider only upto two qubit gates
            num_operands = rng.choice([1, 2], p=[0.5, 0.5]) if max_possible_operands > 1 else 1 # equal probability for single and two qubit gates when maximum_posible_oprerands > 1
            if num_operands == 1: 
                operands,operations = [rng.choice(qubits)], rng.choice(one_q_ops)  
            elif num_operands == 2:
                operations, operands = rng.choice(two_q_ops), list(rng.choice(edges))

            qubits = [q for q in qubits if q not in  operands]
            if edges:
                edges = [pair for pair in edges if pair[0] not in operands and pair[1] not in operands]

            if allow_two_q_op and not edges:
                allow_two_q_op = False
        
            if operations in one_param:
                num_angles = 1
            elif operations in two_param:
                num_angles = 2
            elif operations in three_param:
                num_angles = 3
            else:
                num_angles = 0
            angles = [rng.uniform(0, 2*np.pi) for _ in range(num_angles)]
            op = operations(*angles)    
            qc.append(CircuitInstruction(operation=op,qubits=operands))
        qc.barrier()

    #qc.measure_all() # uncomment it if measurement gate required

    return qc
        

In [5]:
circuit = RQC(4,6,['rz','cx','x','tdg','rx'])
circuit.draw()

┌─────┐     ░    ┌─────┐     ░       ░     ┌───┐            ░ »
q_0: ───┤ Tdg ├─────░────┤ Tdg ├─────░───■───░─────┤ X ├────────────░─»
     ┌──┴─────┴───┐ ░ ┌──┴─────┴───┐ ░ ┌─┴─┐ ░     └─┬─┘    ┌─────┐ ░ »
q_1: ┤ Rz(3.0499) ├─░─┤ Rz(1.7632) ├─░─┤ X ├─░───────┼──────┤ Tdg ├─░─»
     └──┬─────┬───┘ ░ ├────────────┤ ░ └───┘ ░       │      └─────┘ ░ »
q_2: ───┤ Tdg ├─────░─┤ Rx(2.6269) ├─░───■───░───────■──────────────░─»
     ┌──┴─────┴───┐ ░ └───┬───┬────┘ ░ ┌─┴─┐ ░ ┌───────────┐        ░ »
q_3: ┤ Rz(1.4199) ├─░─────┤ X ├──────░─┤ X ├─░─┤ Rz(1.388) ├────────░─»
     └────────────┘ ░     └───┘      ░ └───┘ ░ └───────────┘        ░ »
«        ┌─────┐     ░ ┌────────────┐      ░ 
«q_0: ───┤ Tdg ├─────░─┤ Rz(2.5916) ├──────░─
«        └┬───┬┘     ░ └────────────┘      ░ 
«q_1: ────┤ X ├──────░─────────────────■───░─
«     ┌───┴───┴────┐ ░ ┌────────────┐  │   ░ 
«q_2: ┤ Rx(5.0207) ├─░─┤ Rz(2.1952) ├──┼───░─
«     ├────────────┤ ░ └────────────┘┌─┴─┐ ░ 
«q_3: ┤ Rx(5.4983) ├─░───────────────┤ X ├─░─
«     └────────────┘ ░               └───┘ ░

## Bonus Problem

The Coupling map Given in the bonus section:


![Bonus Problem](cmap.png)

In [6]:
sym_cmap = CouplingMap(couplinglist=[[0,1],[1,0],[1,2],[2,1],[2,3],[3,2],[3,4],[4,3],[4,1],[1,4],[5,4],[4,5]]) 
circuit = RQC(6,4,['rz','cx','x','tdg','rx'],coupling_map=sym_cmap)
circuit.draw()

┌─────┐     ░      ┌───┐                    ░    ┌─────┐     ░ »
q_0: ────┤ Tdg ├─────░──────┤ X ├────────────────────░────┤ Tdg ├─────░─»
         └┬───┬┘     ░      ├───┤                    ░ ┌──┴─────┴───┐ ░ »
q_1: ─────┤ X ├──────░──────┤ X ├────────────────────░─┤ Rz(6.1498) ├─░─»
          └─┬─┘      ░      └─┬─┘     ┌───────────┐  ░ └───┬───┬────┘ ░ »
q_2: ───────■────────░────────┼───────┤ Rz(6.139) ├──░─────┤ X ├──────░─»
     ┌─────────────┐ ░        │       ├───────────┴┐ ░     └─┬─┘      ░ »
q_3: ┤ Rx(0.42271) ├─░────────┼───────┤ Rx(1.6401) ├─░───────■────────░─»
     ├─────────────┤ ░        │       └────────────┘ ░ ┌────────────┐ ░ »
q_4: ┤ Rz(0.23177) ├─░────────■──────────────────────░─┤ Rx(4.2163) ├─░─»
     └┬────────────┤ ░ ┌─────────────┐               ░ └──┬─────┬───┘ ░ »
q_5: ─┤ Rx(5.2537) ├─░─┤ Rz(0.97083) ├───────────────░────┤ Tdg ├─────░─»
      └────────────┘ ░ └─────────────┘               ░    └─────┘     ░ »
«                    ░ 
«q_0: ──────■────────░─
«         ┌─┴─┐      ░ 
«q_1: ────┤ X ├──────░─
«         ├───┤      ░ 
«q_2: ────┤ X ├──────░─
«         └─┬─┘      ░ 
«q_3: ──────■────────░─
«     ┌────────────┐ ░ 
«q_4: ┤ Rz(5.0116) ├─░─
«     ├───────────┬┘ ░ 
«q_5: ┤ Rx(2.482) ├──░─
«     └───────────┘  ░

In [7]:
# defining coupling map of the problem

cmap = CouplingMap(couplinglist=[[0,1],[1,2],[2,3],[3,4],[4,1],[5,4]]) # considering only one way coupling
circuit = RQC(6,3,['rz','cx','x','tdg','rx'],coupling_map=cmap)
circuit.draw()

┌───┐      ░ ┌───────────┐       ░ ┌───┐ ░ 
q_0: ┤ X ├──────░─┤ Rx(1.948) ├───────░─┤ X ├─░─
     ├───┤      ░ └───────────┘ ┌───┐ ░ └───┘ ░ 
q_1: ┤ X ├──────░───────────────┤ X ├─░───■───░─
     └─┬─┘      ░ ┌────────────┐└─┬─┘ ░ ┌─┴─┐ ░ 
q_2: ──┼────■───░─┤ Rx(3.0817) ├──┼───░─┤ X ├─░─
       │  ┌─┴─┐ ░ └───┬───┬────┘  │   ░ ├───┤ ░ 
q_3: ──┼──┤ X ├─░─────┤ X ├───────┼───░─┤ X ├─░─
       │  └───┘ ░     └───┘       │   ░ ├───┤ ░ 
q_4: ──■────────░─────────────────■───░─┤ X ├─░─
          ┌───┐ ░ ┌────────────┐      ░ └─┬─┘ ░ 
q_5: ─────┤ X ├─░─┤ Rz(0.4539) ├──────░───■───░─
          └───┘ ░ └────────────┘      ░       ░

**Refernces**

1. **[Random Quantum Circuit - Qiskit Source Code](https://qiskit.org/documentation/_modules/qiskit/circuit/random/utils.html)**
2. **[RAndom Circuit Block Encoded Matrix (RACBEM)](https://github.com/qsppack/RACBEM)**
3. **[Coupling Map - Discussion](https://quantumcomputing.stackexchange.com/questions/22023/coupling-maps-in-quantum-computation)**